In [2]:
import json

class Phone:
    def __init__(self, number, type):
        self.number = number
        self.type = type

class Contact:
    def __init__(self, id, owner_id, contact_nickname, phones):
        self.id = id
        self.owner_id = owner_id
        self.contact_nickname = contact_nickname
        self.phones = phones

def load_contacts_from_json(file_path):
    contacts = []
    with open(file_path, 'r') as file:
        data = json.load(file)
        for item in data:
            id = item.get('id')
            owner_id = item.get('owner_id')
            contact_nickname = item.get('contact_nickname')
            phones = []
            for phone_data in item.get('phone', []):
                number = normalize_phone(phone_data.get('number'))
                type = phone_data.get('type')
                phone = Phone(number, type)
                phones.append(phone)
            contact = Contact(id, owner_id, contact_nickname, phones)
            contacts.append(contact)
    return contacts

def normalize_phone(phone_number):
    # Remove non-digit characters and country code if present
    digits = ''.join(filter(str.isdigit, phone_number))
    if len(digits) == 11 and digits[0] == '1':
        digits = digits[1:]  # Remove country code '1'
    return digits



In [3]:
file_path = 'contacts.json'
contacts = load_contacts_from_json(file_path)
for contact in contacts:
    print(f"Contact ID: {contact.id}, Owner ID: {contact.owner_id}, Nickname: {contact.contact_nickname}")
    for phone in contact.phones:
        print(f"  Phone Number: {phone.number}, Type: {phone.type}")

Contact ID: 0, Owner ID: 0, Nickname: Alan
  Phone Number: 2123459974, Type: landline
  Phone Number: 5989555120, Type: cell
Contact ID: 1, Owner ID: 1, Nickname: Jane
  Phone Number: 2123458974, Type: cell
Contact ID: 2, Owner ID: 2, Nickname: Steve
  Phone Number: 2123458974, Type: landline
Contact ID: 3, Owner ID: 3, Nickname: Jane
  Phone Number: 2123411113, Type: landline
  Phone Number: 9173454700, Type: cell


In [4]:
from load_persons import load_persons_from_json
persons = load_persons_from_json('persons.json')
contacts = load_contacts_from_json('contacts.json')

In [6]:
def get_phones_by_person(persons, contacts):
    contact_phones_by_person = dict()

    for person_id in persons:
        person = persons[person_id]
        for contact in contacts:
            if(contact.owner_id == person.id):
                contact_phones_by_person[person.id] = (person.phone, [phone.number for phone in contact.phones])

    return contact_phones_by_person


contact_phones_by_person = get_phones_by_person(persons, contacts)
print(contact_phones_by_person)

{0: ('1-2123458974', ['2123459974', '5989555120']), 1: ('(212) 345-8974', ['2123458974']), 2: ('1-2123411113', ['2123458974']), 3: ('(-1) 2-12345897', ['2123411113', '9173454700'])}


In [7]:
def has_phone_in_contacts(phone, contact_phones):
    return phone in contact_phones

In [9]:
def get_connected_persons_by_contact(contact_phones_by_person, person_id):
    connected_ids = []
    
    if person_id not in contact_phones_by_person:
        return []
    
    person_contact_phones = contact_phones_by_person[person_id][1]
    person_phone_number = normalize_phone(contact_phones_by_person[person_id][0])

       
    for target_id in contact_phones_by_person:
        if (target_id != person_id and target_id in contact_phones_by_person ):
            target_contact_phones = contact_phones_by_person[target_id][1]
            target_phone_number = normalize_phone(contact_phones_by_person[target_id][0])
            if has_phone_in_contacts(person_phone_number, target_contact_phones) or has_phone_in_contacts(target_phone_number, person_contact_phones):
                connected_ids.append(target_id)


    
    return connected_ids

print(get_connected_persons_by_contact(contact_phones_by_person, 0))
print(get_connected_persons_by_contact(contact_phones_by_person, 1))
print(get_connected_persons_by_contact(contact_phones_by_person, 2))
print(get_connected_persons_by_contact(contact_phones_by_person, 3))
print(get_connected_persons_by_contact(contact_phones_by_person, 4))
print(get_connected_persons_by_contact(contact_phones_by_person, 5))



[1, 2]
[0, 2]
[0, 1, 3]
[2]
[]
[]
